In [33]:
from skfem import *
import numpy as np
from skfem.visuals.matplotlib import draw, plot
from skfem.utils import solver_iter_krylov
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
from skfem import *
from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.dpi'] = 200

pi = np.pi
sin = np.sin
cos = np.cos
exp = np.exp

## Problem


\begin{aligned}
\left(\nabla w_{h}, \nabla \chi_{h}\right) &=\left(f, \chi_{h}\right) & & \forall \chi_{h} \in W_{h}
\end{aligned}

## Forms and errors

In [34]:
def exact_u(x, y):
    return sin(pi * x) * sin(pi * y)


def dexact_u(x, y):
    dux = pi * cos(pi * x) * sin(pi * y)
    duy = pi * cos(pi * y) * sin(pi * x)
    return dux, duy

In [35]:
@Functional
def L2uError(w):
    x, y = w.x
    return (w.w - exact_u(x, y))**2


def get_DuError(basis, u):
    duh = basis.interpolate(u).grad
    x = basis.global_coordinates().value
    dx = basis.dx  # quadrature weights
    dux, duy = dexact_u(x[0], x[1])
    return np.sqrt(np.sum(((duh[0] - dux)**2 + (duh[1] - duy)**2) * dx))


@LinearForm
def f_load(v, w):
    global ww
    ww = w
    pix = pi * w.x[0]
    piy = pi * w.x[1]
    return (2 * pi**2 * sin(pix) * sin(piy)) * v


@BilinearForm
def laplace(u, v, w):
    return dot(grad(u), grad(v))

In [36]:
m = MeshTri()

m.refine(5)
element = ElementTriP2()
basis = InteriorBasis(m, element, intorder=3)

K1 = asm(laplace, basis)
f1 = asm(f_load, basis)

uh = solve(*condense(K1, f1, D=basis.find_dofs()),
           solver=solver_iter_krylov(tol=1e-8))

U = basis.interpolate(uh).value

L2u = np.sqrt(L2uError.assemble(basis, w=U))
Du = get_DuError(basis, uh)
H1u = Du + L2u
print('L2: ', L2u)
print('H1: ', H1u)

L2:  1.2515205687725662e-05
H1:  0.0013943264128173747


In [39]:
m = MeshTri()

for i in range(1, 11):
    m.refine()
    element = ElementTriP2()
    basis = InteriorBasis(m, element, intorder=3)

    K1 = asm(laplace, basis)
    f1 = asm(f_load, basis)

    uh = solve(*condense(K1, f1, D=basis.find_dofs()),
               solver=solver_iter_krylov(tol=1e-8))
    
    U = basis.interpolate(uh).value

    L2u = np.sqrt(L2uError.assemble(basis, w=U))
    Du = get_DuError(basis, uh)
    H1u = Du + L2u
    print('L2: ', L2u)
    print('H1: ', H1u)
    

L2:  0.04225109196587925
H1:  0.38050952746310146
L2:  0.005987736383244291
H1:  0.09357395999914364
L2:  0.0007858901968989981
H1:  0.022841770498715858
L2:  9.971026877266543e-05
H1:  0.005624103978393136
L2:  1.2515205687725662e-05
H1:  0.0013943264128173747
L2:  1.5660987890845767e-06
H1:  0.0003470663338913969
L2:  1.9581826654013292e-07
H1:  8.657394882554599e-05
L2:  2.4479213716165906e-08
H1:  2.1619207474292677e-05
L2:  3.059986007432025e-09
H1:  5.4017544061605344e-06
L2:  3.828108161424987e-10
H1:  1.3500572023548e-06


In [28]:
element = ElementTriP2()
basis = InteriorBasis(m, element, intorder=3)

In [23]:
basis.doflocs.shape

(2, 25)

In [24]:
basis.get_dofs()

DofsView(obj=<skfem.assembly.dofs.Dofs object at 0x00000178283DDD08>, nodal_ix=array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 16, 17, 18],
      dtype=int64), facet_ix=array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 14, 15, 24, 25, 28, 29],
      dtype=int64), edge_ix=array([], dtype=int64), interior_ix=array([], dtype=int64), nodal_rows=[0], facet_rows=slice(0, 0, None), edge_rows=slice(0, 0, None), interior_rows=slice(0, 0, None))

In [25]:
m.boundary_nodes()

array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 16, 17, 18],
      dtype=int64)

In [26]:
m.boundary_facets()

array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 14, 15, 24, 25, 28, 29],
      dtype=int64)

In [29]:
basis.doflocs.shape

(2, 81)

In [30]:
basis.get_dofs()

DofsView(obj=<skfem.assembly.dofs.Dofs object at 0x000001783FD37A88>, nodal_ix=array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 16, 17, 18],
      dtype=int64), facet_ix=array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 14, 15, 24, 25, 28, 29],
      dtype=int64), edge_ix=array([], dtype=int64), interior_ix=array([], dtype=int64), nodal_rows=[0], facet_rows=[0], edge_rows=slice(0, 0, None), interior_rows=slice(0, 0, None))

In [31]:
m.boundary_nodes()

array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 14, 16, 17, 18],
      dtype=int64)

In [32]:
m.boundary_facets()

array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 14, 15, 24, 25, 28, 29],
      dtype=int64)